In [1]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00


In [2]:
import os
import shutil
import yaml
import glob
from ultralytics import YOLO

# ==========================================
# 1. SETUP: COPY DATA TO WRITABLE FOLDER
# ==========================================
INPUT_DIR = '/kaggle/input/ppeconstruction' 
WORKING_DIR = '/kaggle/working/ppe_data'

print("🔄 Copying dataset to writable directory...")

# Clean up previous runs
if os.path.exists(WORKING_DIR):
    shutil.rmtree(WORKING_DIR)

# Copy folder
shutil.copytree(INPUT_DIR, WORKING_DIR)
print(f"✅ Dataset copied to: {WORKING_DIR}")

# ==========================================
# 2. CLEAN LABELS (Remove 'No-Boots')
# ==========================================
print("🧹 Cleaning 'No-Boots'...")

# Find the YAML file
yaml_files = glob.glob(os.path.join(WORKING_DIR, '*.yaml'))
if not yaml_files:
    # Sometimes it's inside a subfolder, let's search recursively just in case
    yaml_files = glob.glob(os.path.join(WORKING_DIR, '**/*.yaml'), recursive=True)

if not yaml_files:
    raise FileNotFoundError("❌ No .yaml file found in dataset!")

yaml_file = yaml_files[0]
print(f"   Found config: {yaml_file}")

target_class_id = -1

with open(yaml_file, 'r') as f:
    data = yaml.safe_load(f)

# A. Find Class ID
raw_names = data['names']
names_dict = {i: n for i, n in enumerate(raw_names)} if isinstance(raw_names, list) else raw_names

for cid, cname in names_dict.items():
    if 'boot' in str(cname).lower() and 'no' in str(cname).lower():
        target_class_id = cid
        print(f"   🎯 Removing Class ID {cid}: {cname}")
        break

# B. Remove from Text Files
if target_class_id != -1:
    txt_files = glob.glob(f'{WORKING_DIR}/**/*.txt', recursive=True)
    for txt in txt_files:
        with open(txt, 'r') as f: lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.split()
            if not parts: continue
            cls = int(parts[0])
            if cls == target_class_id: continue 
            if cls > target_class_id: parts[0] = str(cls - 1) 
            new_lines.append(" ".join(parts) + "\n")
        with open(txt, 'w') as f: f.writelines(new_lines)

    # Update Names
    new_names = {i: n for i, n in enumerate([v for k,v in names_dict.items() if k != target_class_id])}
    data['names'] = new_names
    data['nc'] = len(new_names)

# ==========================================
# 3. FIX PATHS (Crucial for your structure)
# ==========================================
# Your structure is: /kaggle/working/ppe_data/images/train
# So 'path' = /kaggle/working/ppe_data
# And 'train' = images/train

data['path'] = WORKING_DIR  # Absolute Root
data['train'] = 'images/train'
data['val'] = 'images/val'   # Usually 'valid' or 'val', checking...
data['test'] = 'images/test'

# Verify if 'val' folder is named 'valid' instead
if os.path.exists(os.path.join(WORKING_DIR, 'images/valid')):
    data['val'] = 'images/valid'

if 'download' in data: del data['download']

# Save Fixed YAML
with open(yaml_file, 'w') as f:
    yaml.dump(data, f)

print(f"✅ YAML Paths Fixed: train={data['train']}, val={data['val']}")

# ==========================================
# 4. TRAIN (2x GPU)
# ==========================================
print("🔥 Starting YOLOv11m Training...")

model = YOLO('yolo11m.pt')

results = model.train(
    data=yaml_file,
    # --- The "Winning Recipe" ---
    epochs=300,                # Paper used long training (stops early if needed)
    patience=50,               # Stop if no improvement for 50 epochs
    batch=32,                  # High batch size for stable gradients
    imgsz=640,
    
    # Optimizer Settings (Crucial for 95% mAP)
    optimizer='SGD',           # Research standard (often beats Adam for YOLO)
    lr0=0.01,                  # Initial Learning Rate
    lrf=0.01,                  # Final Learning Rate (Cosine decay)
    momentum=0.937,            # High momentum
    weight_decay=0.0005,       # Regularization to prevent overfitting
    
    # Augmentation (To fix your class imbalance)
    augment=True,
    mosaic=1.0,                # 100% Mosaic (Essential for small objects like goggles)
    mixup=0.15,                # 15% Mixup (Fixes the "No-Helmet" imbalance)
    copy_paste=0.1,            # 10% Copy-Paste (Standard in 2024 papers)
    
    # Hardware
    device=[0, 1],             # Use both Kaggle GPUs
    project='/kaggle/working/runs',
    name='paper_replication_run'
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔄 Copying dataset to writable directory...
✅ Dataset copied to: /kaggle/working/ppe_data
🧹 Cleaning 'No-Boots'...
   Found config: /kaggle/working/ppe_data/data.yaml
   🎯 Removing Class ID 10: no_boots
✅ YAML Paths Fixed: train=images/train, val=images/val
🔥 Starting YOLOv11m Training...
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode

In [3]:
!zip -r sgd_trained_yolo11m.zip /kaggle/working/runs

  adding: kaggle/working/runs/ (stored 0%)
  adding: kaggle/working/runs/paper_replication_run/ (stored 0%)
  adding: kaggle/working/runs/paper_replication_run/BoxR_curve.png (deflated 6%)
  adding: kaggle/working/runs/paper_replication_run/weights/ (stored 0%)
  adding: kaggle/working/runs/paper_replication_run/weights/last.pt (deflated 8%)
  adding: kaggle/working/runs/paper_replication_run/weights/best.pt (deflated 8%)
  adding: kaggle/working/runs/paper_replication_run/val_batch1_labels.jpg (deflated 12%)
  adding: kaggle/working/runs/paper_replication_run/args.yaml (deflated 53%)
  adding: kaggle/working/runs/paper_replication_run/val_batch0_pred.jpg (deflated 12%)
  adding: kaggle/working/runs/paper_replication_run/val_batch0_labels.jpg (deflated 12%)
  adding: kaggle/working/runs/paper_replication_run/val_batch1_pred.jpg (deflated 11%)
  adding: kaggle/working/runs/paper_replication_run/train_batch0.jpg (deflated 2%)
  adding: kaggle/working/runs/paper_replication_run/confusion_

In [4]:
!pip install ipython

In [5]:
from IPython.display import FileLink

In [6]:
FileLink('sgd_trained_yolo11m.zip')

/kaggle/working/sgd_trained_yolo11m.zip